# VOLTANDO UMA PASTA PARA TRAZ

In [1]:
import os
os.chdir("../")

# DOWNLOAD DOS ARQUIVOS

In [ ]:
from modules.ComexStatDownloader import ComexStatDownloader

CSD = ComexStatDownloader()
CSD.download_files()

# CONEXÃO COM BANCO

In [ ]:
from modules.Database.postgres_connection import PostgresConnection
from modules.Database.database_manager import DatabaseManager

host    = "localhost"
db      = "comexstat"
user    = "postgres"
pwd     = "postgres"  # Substitua pela sua senha real

# Certifique-se de definir as variáveis host, db, user, pwd antes de usar
conn = PostgresConnection(host, db, user, pwd)
conn.connect()
db_manager = DatabaseManager(conn)

# Cria as tabelas no banco de dados
with open("repository\\querys\\create_table.sql", "r") as file:
    create_tables_sql = file.read()
    db_manager.execute_raw(create_tables_sql)

# Insere os dados no banco de dados
base_paths = ["data\\external\\mun", "data\\external\\ncm"]

for file in os.listdir("repository\\data"):
    if file.endswith(".csv"):
        file_path = os.path.join("repository\\data", file)
        with open(file_path, "r") as f:
            insert_sql = f.read()
            db_manager.execute_raw(insert_sql)